# Setup

Der Code unterhalb braucht nur gensim, kann mit `pip install gensim` installiert werden. 

In [1]:
import os
import xml.etree.ElementTree as ET
from gensim.models import Word2Vec
from gensim.models import FastText
import subprocess
import re


# Word2Vec trainieren

Der Code unterhalb ist nur relevant, wenn du das Modell neu trainieren willst - ein fertiges Modell ist allerdings schon geuploaded. 

Der Code zum Testen / Anwenden ist weiter unten. 

In [ ]:
def cleanup(text:str, re_cleanup:bool, lowercase:bool)->str: 
    if lowercase: 
            text = text.lower()
    if re_cleanup: 
        # Replace all punctuation chars with a single whitespace, keep only normal words and digits
        text = re.sub(r"[^\w\n\d]+", ' ', text)
        text = re.sub(r"^\s+|\s+(?=\n)", '', text, flags=re.MULTILINE) # Delete leading/trailing whitespaces
    text = re.sub(r'[^(\n|\S)]+', ' ', text) # Clean up: spaces (except newlines) to single space
    return text


def extract_text_from_xml(file_path):
    try:
        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Ensure the XML has the expected format (root is <body>)
        if root.tag != 'body':
            return []

        # Extract text content from all <text> elements
        text_elements = root.findall('.//text')
        texts = [cleanup(elem.text, re_cleanup=True, lowercase=True) for elem in text_elements if elem.text]
        print(f"Successfully read file {file_path}")
        return texts
    except Exception as e:
        print(f"Error when reading file {file_path}")
        return []


def process_folder(folder_path):
    all_texts = []

    # Walk through all files and subdirectories
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check if the file is an .xml file
            if file.endswith('.xml'):
                file_path = os.path.join(root, file)
                # Extract text from the XML file
                extracted_texts = extract_text_from_xml(file_path)
                all_texts.extend(extracted_texts)

    return all_texts

# Specify the folder to process
folder_path = 'XML'

# Process the folder and print the results
all_extracted_texts = process_folder(folder_path)

Successfully read file XML/touchepasamastatue_output.xml
Error when reading file XML/._touchepasamastatue_output.xml
Successfully read file XML/Hélix_output.xml
Error when reading file XML/._novelumcarcassonne_output.xml
Successfully read file XML/maquis_output.xml
Error when reading file XML/._normaux_output.xml
Successfully read file XML/meduanocta_output.xml
Error when reading file XML/._Nemesis2_output.xml
Successfully read file XML/braves_output.xml
Error when reading file XML/._Hélix_output.xml
Error when reading file XML/._Mora_output.xml
Error when reading file XML/._Destoursetdeslys_output.xml
Successfully read file XML/Destoursetdeslys_output.xml
Error when reading file XML/._meduanocta_output.xml
Successfully read file XML/Alvarium_output.xml
Successfully read file XML/Nemesis2_output.xml
Successfully read file XML/normaux_output.xml
Successfully read file XML/tenesoun_output.xml
Error when reading file XML/._Nemesis_output.xml
Successfully read file XML/novelumcarcassonne_o

In [40]:
with open("training_corpus.txt", 'w') as fw: 
    fw.writelines('\n'.join(all_extracted_texts))

In [ ]:
FastText()

In [52]:
# Train Model
model = FastText(
    corpus_file="training_corpus.txt",
    vector_size=300,  # Dimensionality of the word embeddings
    window=5,         # Max distance between current and predicted word
    min_count=3,      # Ignores all words with total frequency < 3
    workers=4,        # Number of threads to run in parallel
    epochs=5,        # Number of training epochs
    sg=1
)

# 4) Save the trained model for later use
model.save("word2vec.model")

KeyboardInterrupt: 

In [39]:
# Assume `model` is your trained Word2Vec model
vocabulary = model.wv.key_to_index

# Print the vocabulary (all words)
with open("vocabulary.txt", 'w') as fw: 
    fw.writelines('\n'.join(list(vocabulary.keys())))

# Modell anwenden

Wenn in fastText-Modell verwendet wird (ist beim aktuellen word2vec-Modell und dem zugehörigen Code oberhalb der Fall), können beliebige Sequenzen und Wörter getestet werden, unabhängig davon, ob diese in den Trainingsdaten waren. In diesem Fall wird die Eingabe in einzelne, kleinere Buchstabenfolgen zerlegt, die sich in den Trainingsdaten finden. Trotzdem ist das Ergebnis auf Wörtern aus dem Vokabular (kann mit dem Code direkt oberhalb in eine Datei geschrieben werden) vermutlich ab besten. 

Wenn ein normales Word2Vec-Modell verwendet wird (nicht empfohlen, schlechtere Ergebnisse), können nur Wörter aus dem Vokabular eingegeben werden.

In [21]:
# Load Model
model = FastText.load("word2vec.model")

In [ ]:
# Find nearest neighbours for single word

similar_words = model.wv.most_similar("arme", topn=20)
print(similar_words)

[('alarme', 0.8317347764968872), ('vacarme', 0.7709546685218811), ('armel', 0.7630341649055481), ('larme', 0.7539097666740417), ('parme', 0.7298886179924011), ('carmel', 0.6511124968528748), ('armenie', 0.6490821838378906), ('ariège', 0.6351330876350403), ('handicapée', 0.6339969635009766), ('envergure', 0.6334922909736633), ('bolide', 0.6313551664352417), ('algorithme', 0.6310719847679138), ('enorme', 0.6272657513618469), ('aveugle', 0.6268576979637146), ('angoisse', 0.6240061521530151), ('pénitentiaire', 0.6237596869468689), ('enferme', 0.6236466765403748), ('suicidaire', 0.623491644859314), ('arène', 0.6218565106391907), ('allègre', 0.6210903525352478)]


In [32]:
# Check similarity between two words
similarity = model.wv.similarity("aujourd'hui", "a")
print(similarity)


-0.030709
